In [ ]:
import pandas as pd
from datetime import time
import numpy as np

In [ ]:
patient_info=pd.read_csv('AsthmaFiles/patient_info.csv')
smartwatch1=pd.read_csv('AsthmaFiles/smartwatch1.csv')
smartwatch2=pd.read_csv('AsthmaFiles/smartwatch2.csv')
smartwatch3=pd.read_csv('AsthmaFiles/smartwatch3.csv')

In [ ]:
patient_info[["user_key"]+list(patient_info)[-11:-2]]

In [ ]:
list_of_user_keys=[]
for x in patient_info[["user_key"]+list(patient_info)[-10:-2]].iterrows():
    if x[1]["pef_end_date"] - x[1]["pef_start_date"] >= 50 and x[1]["miband_end_date"]!="NaN":
        list_of_user_keys.append(x[1]["user_key"])
list_of_user_keys

In [ ]:
def get_peakflow_data(user_key):
    peakflow_data = pd.read_csv("AsthmaFiles/peakflow.csv")
    peakflow_data = peakflow_data[peakflow_data["user_key"]==user_key]
    peakflow_data = peakflow_data[["date","hour","pef_max"]]
    return peakflow_data


In [ ]:
def pair_weather(id,dates):
    weather = pd.read_csv("AsthmaFiles/environment.csv")
    for_id = weather.loc[weather['user_key'] == id]
    weather = for_id.loc[for_id['date'].isin(dates)]

    return weather

In [ ]:
peak_flow_data = get_peakflow_data(list_of_user_keys[0])
weather = pair_weather(list_of_user_keys[0],peak_flow_data["date"])
weather

In [ ]:
def try_catch(row,default,peak_flow_data,weather,x):
    try:
        return weather.loc[weather['date'] == row["date"]].iloc[0][x]
    except IndexError:
        return default

In [ ]:

# seperate_for_key = smartwatch1.loc[smartwatch1["user_key"]==list_of_user_keys[0]][["date","time","hr"]]
# seperate_for_date = seperate_for_key.loc[seperate_for_key["date"]==1][["time","hr"]]
# seperate_for_time = seperate_for_date.loc[seperate_for_date["time"].str.startswith("08")]["hr"]

def seperate_for_key(user_key,smartwatch1,smartwatch2,smartwatch3):
    seperate_for_key1 = smartwatch1.loc[smartwatch1["user_key"]==user_key][["date","time","hr"]]
    seperate_for_key2 = smartwatch2.loc[smartwatch2["user_key"]==user_key][["date","time","hr"]]
    seperate_for_key3 = smartwatch3.loc[smartwatch3["user_key"]==user_key][["date","time","hr"]]
    frames = [seperate_for_key1, seperate_for_key2, seperate_for_key3]
    return pd.concat(frames)

def seperate_for_date(date,seperate_for_key):
    return seperate_for_key.loc[seperate_for_key["date"]==date][["time","hr"]]

def seperate_for_time(time,seperate_for_date):
    if len(str(time)) == 1:
        time = "0" + str(time)
    return seperate_for_date.loc[seperate_for_date["time"].str.startswith(str(time))]["hr"].max()


def confusing_method(row,key,sm1,sm2,sm3):
    sk = seperate_for_key(key,sm1,sm2,sm3)
    sd = seperate_for_date(row["date"],sk)
    return seperate_for_time(row["hour"],sd)



In [ ]:
default = "NA"
for i in list_of_user_keys:
    peak_flow_data = get_peakflow_data(i)
    weather = pair_weather(i,peak_flow_data["date"])
    for x in list(weather.columns):
        if x not in peak_flow_data.columns:
            peak_flow_data[x] = peak_flow_data.apply(lambda row: try_catch(row,default,peak_flow_data,weather,x), axis = 1)
    peak_flow_data["hr"] = peak_flow_data.apply(lambda row: confusing_method(row,i,smartwatch1,smartwatch2,smartwatch3), axis = 1)
    peak_flow_data.to_csv(f"AsthmaFiles/{int(i)}.csv",index=False)

In [ ]:

import pandas as pd

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)


dataset_190= pd.read_csv("AsthmaFiles/190.csv")
z = dataset_190["pef_max"].max()
# z = 575
dataset_190 = dataset_190.drop(["weed_pollen","tree_pollen","grass_pollen"],axis=1)
dataset_190=clearned_dataset_190 = clean_dataset(dataset_190)
dataset_190
y = dataset_190["pef_max"].apply(lambda x: x/z)
x = dataset_190.drop(["pef_max","date","hour","user_key","co","no","no2","o3","so2","pm2_5","pm10","nh3"],axis=1)
x

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=74)


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x_train,y_train)


In [ ]:

y_pred = model.predict(x_test)
y_pred

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming you have y_pred and y_test as NumPy arrays or Pandas Series
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R-squared (R^2) Score: {r2}')

